In [50]:
from sklearn.preprocessing import LabelEncoder


# Apply the mapping to both the training and test data
def map_ordinal_features(df, mappings):
    for feature, mapping in mappings.items():
        df[feature] = df[feature].map(mapping)
    return df

def frequency_encoding(train_df, test_df, column):
    # Get the frequency of each category in the train data
    freq_map = train_df[column].value_counts(normalize=True).to_dict()

    # Map the frequencies to the train and test data using the same mapping
    train_df[column] = train_df[column].map(freq_map)
    test_df[column] = test_df[column].map(freq_map)

    return train_df, test_df

def binary_encoding(df, mappings):
    for feature, mapping in mappings.items():
        df[feature] = df[feature].map(mapping)
    return df

#Titanic Survival Prediction

In [51]:
# Clone the GitHub repository
!git clone https://github.com/geopan2000/TitanicSurvival.git

# Change directory to the cloned repository
%cd TitanicSurvival

Cloning into 'TitanicSurvival'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (12/12), 36.15 KiB | 974.00 KiB/s, done.
/content/TitanicSurvival/TitanicSurvival/TitanicSurvival/TitanicSurvival/TitanicSurvival/TitanicSurvival/TitanicSurvival/TitanicSurvival/TitanicSurvival


In [52]:
import pandas as pd

In [53]:
train_data = pd.read_csv('data/train.csv')
print(train_data.info())
test_data = pd.read_csv('data/test.csv')
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pcl

In [54]:
train_data['Age']=train_data['Age'].fillna(train_data['Age'].mean())
train_data['Age']=train_data['Age'] / 100

test_data['Age']=test_data['Age'].fillna(train_data['Age'].mean())
test_data['Age']=test_data['Age'] / 100

In [55]:
# Apply one-hot encoding on train and test data
train_data = pd.get_dummies(train_data, columns=['Embarked'])
test_data = pd.get_dummies(test_data, columns=['Embarked'])

In [57]:
print(train_data.shape)
print(test_data.shape)

(418, 13)
(891, 14)
